In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

import torch
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note Embedding

In [ ]:
# NEM
note_labels_num = 21
note_tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")
nem = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=note_labels_num).to(device)
state_dict = torch.load("models/NEM.pth", map_location=device)
nem.load_state_dict(state_dict)

In [ ]:
# Note embedding dictionary
df_ne = pd.read_csv("data/note_embedding.csv")
ne_list = df_ne.iloc[:, 1:].values.astype(float)

In [ ]:
def note2vec(note):
    # 将香材文本转为向量
    nem.eval()
    with torch.no_grad():
        encoding = note_tokenizer(
            note,
            truncation=True,
            padding='max_length', 
            max_length=64,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids'].to(device)
        attention_mask = encoding['attention_mask'].to(device)
        outputs = nem.bert(input_ids=input_ids, attention_mask=attention_mask)
        embedding = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    
    # 推荐近似香材
    embedding = embedding.reshape(1, -1)
    similarities = cosine_similarity(embedding, ne_list)[0]
    idx = similarities.argmax()
    rc_vec = ne_list[idx]
    rc_name = df_ne.iloc[idx, 0]
    rc_similarity = similarities[idx]
    return rc_vec, rc_name, rc_similarity

In [ ]:
def get_note_list_vec(note_list):
    vec_list = []
    for note in note_list:
        vec, name, similarity = note2vec(note)
        vec_list.append(vec)
        print(f"Note: {note} => {name} (similarity: {similarity:.4f})")
    return vec_list

# Perfume Vector

In [ ]:
# Get perfume vector
def perfume2vec(top_notes, mid_notes, base_notes):
    top_vector = np.array(get_note_list_vec(top_notes))
    mid_vector = np.array(get_note_list_vec(mid_notes))
    base_vector = np.array(get_note_list_vec(base_notes))
    perfume_vec = top_vector * 0.3 + mid_vector * 0.4 + base_vector * 0.3
    return perfume_vec

# Recommend Perfumes

In [ ]:
# Load perfume vector list
df_pe = pd.read_csv("data/perfume_embedding_wa.csv")
pe_list = df_pe.iloc[:, 1:].values.astype(float)

In [ ]:
def get_recommended_perfumes(top_notes, mid_notes, base_notes, top_k=5):
    perfume_vec = perfume2vec(top_notes, mid_notes, base_notes).reshape(1, -1)
    similarities = cosine_similarity(perfume_vec, pe_list)[0]
    df_top = df_pe.assign(similarity=similarities).nlargest(top_k, 'similarity')
    recommendation = df_top[['name','similarity']].values.tolist()
    return recommendation

In [ ]:
like_top_notes = ["橙子"]
like_mid_notes = ["咖啡"]
like_base_notes = ["巧克力"]
top_k = 10

rc_perfumes = get_recommended_perfumes(like_top_notes, like_mid_notes, like_base_notes, top_k)
display(rc_perfumes)

# Perfume Detail

In [ ]:
# Show perfume details
df_perfume = pd.read_csv("data/1976_clean.csv")
rc_names = [item[0] for item in rc_perfumes]
df_rc_perfume = df_perfume[df_perfume['name'].isin(rc_names)]
df_rc_perfume = df_rc_perfume.set_index('name').loc[rc_names].reset_index()
df_rc_perfume.to_csv("recommended_perfumes_wa.csv", index=False)
display(df_rc_perfume)